In [9]:
import asyncio
import aiohttp
import time

models = [
    [{
        "config": {
            "id": "model_1",
            "hyperparameters": {"fit_intercept": True}
        },
        "X": [[1., 2.], [3., 4.]],
        "y": [5., 6.]
    }],
    [{
        "config": {
            "id": "model_2",
            "hyperparameters": {"fit_intercept": False}
        },
        "X": [[2., 3.], [4., 5.]],
        "y": [8., 10.]
    }]
]

async def train_model(session, model):

    await asyncio.sleep(60)

    async with session.post(r"http://127.0.0.1:8000/fit", json=model) as resp:
        if resp.status == 201:
            data = await resp.json()
            print(f"Model {model[0]['config']['id']} trained successfully: {data}")
        elif resp.status == 422:
            print("Invalid input")
        else:
            print("Server Error")

async def load_model(session, model_id):
    json = {"id": model_id}
    async with session.post(r"http://127.0.0.1:8000/load", json=json) as resp:
        if resp.status == 200:
            data = await resp.json()
            print(f"Model {model_id} loaded successfully: {data}")
        elif resp.status == 422:
            print("Invalid input")
        else:
            print("Server Error")

async def predict(session, model_id, X):
    json = {"id": model_id, "X": X}
    async with session.post(r"http://127.0.0.1:8000/predict", json=json) as resp:
        if resp.status == 200:
            data = await resp.json()
            print(f"Prediction from model {model_id}: {data}")
        elif resp.status == 422:
            print("Invalid input")
        else:
            print("Server Error")

async def list_models(session: aiohttp.ClientSession):
    async with session.get(r"http://127.0.0.1:8000/list_models") as resp:
        if resp.status == 200:
            data = await resp.json()
            print("List of models:", data)
        else:
            print("Server Error")

async def remove_all(session: aiohttp.ClientSession):
    async with session.delete(r"http://127.0.0.1:8000/remove_all") as resp:
        if resp.status == 200:
            data = await resp.json()
            print("All models removed:", data)
        else:
            print("Server Error")

async with aiohttp.ClientSession() as session:
    
    time_start_training = time.time()

    await asyncio.gather(
        train_model(session, models[0]),
        train_model(session, models[1])
    )

    time_end_training = time.time()
    print(f"Total training time: {time_end_training - time_start_training}")
    
    await load_model(session, "model_1")
    
    await asyncio.gather(
        predict(session, "model_1", [[4.0, 5.0]]),
        predict(session, "model_1", [[10.0, 20.0]])
    )

    await list_models(session)

    await remove_all(session)

Model model_1 trained successfully: [{'message': 'Model model_1 trained and saved'}]
Model model_2 trained successfully: [{'message': 'Model model_2 trained and saved'}]
Total training time: 60.0239143371582
Model model_1 loaded successfully: [{'message': 'Model model_1 loaded'}]
Prediction from model model_1: [{'predictions': [11.75]}]
Prediction from model model_1: [{'predictions': [6.5]}]
List of models: [{'models': [{'id': 'model_1'}, {'id': 'model_2'}]}]
All models removed: [{'message': 'Model model_1 removed'}, {'message': 'Model model_2 removed'}]


In [10]:
async with aiohttp.ClientSession() as session:
    
    time_start_training = time.time()
    
    # Теперь сделаем синхронный запрос
    for model in models:
        await train_model(session, model)
        
    time_end_training = time.time()
    print(f"Total training time: {time_end_training - time_start_training}")

    await load_model(session, "model_1")
    
    await asyncio.gather(
        predict(session, "model_1", [[4.0, 5.0]]),
        predict(session, "model_1", [[10.0, 20.0]])
    )

    await list_models(session)

    await remove_all(session)

Model model_1 trained successfully: [{'message': 'Model model_1 trained and saved'}]
Model model_2 trained successfully: [{'message': 'Model model_2 trained and saved'}]
Total training time: 120.02970314025879
Model model_1 loaded successfully: [{'message': 'Model model_1 loaded'}]
Prediction from model model_1: [{'predictions': [6.5]}]
Prediction from model model_1: [{'predictions': [11.75]}]
List of models: [{'models': [{'id': 'model_1'}, {'id': 'model_2'}]}]
All models removed: [{'message': 'Model model_1 removed'}, {'message': 'Model model_2 removed'}]
